In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet

In [2]:
def create_cliente_activo(cliente_activo: SQLPartitionedDataSet,
                          date: str) -> pd.DataFrame:
    """Creates master table with features related to EoP state of customers for one period of data

    Parameters
    ----------
    cliente_activo:
        dataset defined in ´catalog.yml´ - list of active customers at EoP for the given period
    date:
        period to process
    
    Returns
    -------
        Mastertable with information of clientes at EoP
    """

    # Initialize logger
    log = initialize_logger()

    # Load active clientes for period
    log.info(f"Creating cliente_activo...")
    period_to_load = get_previous_month(date)
    df_clientes_activos = cliente_activo.filter_by(date=period_to_load)

    # Return
    return df_clientes_activos

In [3]:
num_vars = ["MONTO", "BALANCE_BROUGHT_FORWARD"]

vars_to_group_by = ["CUSTOMER_ID"]

id_cols = ["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]

past_periods = [1, 2, 3, 4, 5, 6]



In [4]:
def create_master_eventos_fact(eventos_facturados: SQLPartitionedDataSet,
                               cliente_activo: pd.DataFrame,
                               parameters: Dict,
                               date: str) -> pd.DataFrame:
    """Creates master table with billing features for one period of data
    Parameters
    ----------
    eventos_facturados:
        dataset defined in ``catalog_raw.yml`` with raw data information related to customer billing state at EoP
    cliente_activo:
        dataset defined in ``catalog_raw.yml`` with raw data information related to active clients at EoP
    date:
        period to process
    parameters:
        set of project parameters defined in ``parameters.yml``
    Returns
    -------
    pd.DataFrame
        Master table with billing features for one period
    """

    # Initialize logger
    log = initialize_logger()

    write_to_parquet = parameters["write_to_parquet"]
    table_name = "eventos_facturados"
    overwrite = parameters["masters"][table_name]["overwrite"]

    # Check if table was already created
    files = get_mastertable_paths_by_period(parameters=parameters, period=date)
    match = [str(file) for file in files if table_name in file]

    if len(match) > 0 and overwrite is False:
        # If table is found, read parquet:
        log.info(f"Reading {match[0]} table")
        df_ef_past = pd.read_parquet(match[0], engine="pyarrow")

    else:
        # Read parameters
        look_back_months = parameters["masters"]["global"]["look_back_months"]
        periods_to_load = get_last_k_periods(date, look_back_months)
        periods_to_load = tuple(periods_to_load)
        start_date = periods_to_load[-1]
        period_to_load = get_previous_month(date)

        # Calculate period to load for active clients
        df_clientes = cliente_activo[["CUSTOMER_ID"]]
        df_ef = eventos_facturados.filter_by_period(date=periods_to_load).drop_duplicates()
        log.info("Merging with EOP...")
        df_ef = pd.merge(df_clientes,
                         df_ef,
                         on=["CUSTOMER_ID"],
                         how="inner",
                         validate="1:m")

        for var in ["MONTO", "BALANCE_BROUGHT_FORWARD"]:
            log.info(f"----{var}")
            df_ef[var] = scale_values(df=df_ef,
                                      vars_to_groupby=["DATE_EXP"],
                                      var_to_scale=var,
                                      by_ranking=False)

        df_ef_past = add_relative_calculate_past(df_ef,
                                                 id_cols=["CUSTOMER_ID"],
                                                 date_col="DATE_EXP",
                                                 start_date=start_date,
                                                 end_date=period_to_load,
                                                 periods=past_periods,
                                                 period_freq="M",
                                                 agg={'MONTO': [np.nanmean, np.nanmax, np.nanmin],
                                                      'BALANCE_BROUGHT_FORWARD': [np.nanmean, np.nanmax, np.nanmin]}
                                                 )

        # Add date variables
        df_ef_past["DATE_EXP"] = period_to_load

        # Change variable names 
        table_preffix = parameters["masters"]["eventos_facturados"]["table_preffix"]
        rename_table(df_ef_past,
                     preffix=table_preffix,
                     ids_to_exclude=id_cols)
        log.info(f"Exporting {df_ef_past.shape[0]} rows and {df_ef_past.shape[1]} columns")

        if write_to_parquet:
            log.info(f"\n------ Writing {table_name} for period {date} to parquet ------")
            file = f"{parameters['paths']['master_path']}master_{table_name}/master_{table_name}_{date}.parquet"
            df_ef_past.to_parquet(file, engine="pyarrow")

        log.info(f"Exporting {df_ef_past.shape[0]} rows and {df_ef_past.shape[1]} columns")

    return df_ef_past

In [5]:
cliente_activo=catalog.load("cliente_activo")
eventos_facturados=catalog.load("eventos_facturados")

2020-12-30 18:47:31,303 - kedro.io.data_catalog - INFO - Loading data from `cliente_activo` (SQLPartitionedDataSet)...
2020-12-30 18:47:31,305 - kedro.io.data_catalog - INFO - Loading data from `eventos_facturados` (SQLPartitionedDataSet)...


In [8]:
dates = calculate_dates_to_process_for_master(parameters, table_name="eventos_facturados")
print(dates)

['20181203', '20181231', '20190128', '20190225', '20190325', '20190422']


In [9]:
for date in dates:
    print(f"Processing date {date}")
    cliente_activo_df= create_cliente_activo(cliente_activo,date)
    create_master_eventos_fact(eventos_facturados,cliente_activo_df,parameters,date)
    

Processing date 20181203
2020-12-30 18:47:59,912 - aa_engine_pkg.assets.utils.utilities - INFO - Creating cliente_activo...
select distinct CUSTOMER_ID from stg_uy_customer_status where UPPER(STATUS) LIKE '%ACTIVO%' and DATE_EXP = 201811


/u01/miniconda3/envs/uypo37/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1381: SAWarning: Oracle version (19, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))


select * from stg_uy_evento_facturado where DATE_EXP in ('201812', '201811', '201810', '201809', '201808', '201807')
2020-12-30 18:48:26,631 - aa_engine_pkg.assets.utils.utilities - INFO - Merging with EOP...
2020-12-30 18:48:27,098 - aa_engine_pkg.assets.utils.utilities - INFO - ----MONTO
2020-12-30 18:48:27,564 - aa_engine_pkg.assets.utils.utilities - INFO - ----BALANCE_BROUGHT_FORWARD
2020-12-30 18:48:28,007 - aa_engine_pkg.assets.utils.utilities - INFO - Adding relative date between 201807 and 201811
2020-12-30 18:48:28,441 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 1
2020-12-30 18:48:28,610 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 2
2020-12-30 18:48:28,806 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 3
2020-12-30 18:48:29,002 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 4
2020-12-30 18:48:29,229 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding 